In [1]:
import cvxpy as cp
import numpy as np
from beamforming import solve_beamforming_with_selected_antennas



def as_omar(H, max_ant=5):
    lmbda_lb = 1e-6
    lmbda_ub = 100
    # global lmbda_lb, lmbda_ub
    N,K = H.shape

    # step 1
    u = np.zeros((N,1))
    u_new = np.ones((N,1))
    r = 0
    max_iter = 15
    while np.linalg.norm(u-u_new)>0.001 and r < max_iter:
        r += 1
        u = u_new.copy()
        W, _ = sparse_iteration(H, u)
        a = np.linalg.norm(W, axis=1)
        mask = (a>0.01)*1
        if mask.sum()<= max_ant:
            break
        u_new = 1/(np.linalg.norm(W, axis=1) + 1e-5)
    prelim_mask = mask.copy()

    # step 2
    r = 0
    max_iter = 50
    while mask.sum() != max_ant and r < max_iter:
        r += 1
        # if mask.sum() < max_ant:
        lmbda = lmbda_lb + (lmbda_ub - lmbda_lb)/2
        W, _ = sdp_omar(H, lmbda, u_new)
        a = np.linalg.norm(W, axis=1)
        mask = (a>0.01)*1
        print('iteration {}'.format(r), lmbda, mask.sum(), lmbda_lb, lmbda_ub)
        if mask.sum() == max_ant:
            break
        elif mask.sum() > max_ant:
            lmbda_lb = lmbda
        elif mask.sum() < max_ant:
            lmbda_ub = lmbda
    if mask.sum()>max_ant:
        mask = prelim_mask.copy()    
    print('num selected antennas', mask.sum())

    # step 3
    W, obj = solve_beamforming_with_selected_antennas(H, mask)
    print(obj)
    return obj.copy(), mask.copy()

def sparse_iteration(H, u, M=1000, noise_var=1, min_snr=1):
    """
    Solves the relaxed formulation of Omar et al 2013
    """
    # print('z mask: {},\n z value: {}'.format(z_mask, z_sol))
    N, K = H.shape
    W = cp.Variable((N,K), complex=True)

    obj = cp.Minimize((u.T @ cp.norm_inf(W, axis=1)))

    zero = np.zeros(N)
    one = np.ones(N)
    c_1 = (1/np.sqrt(min_snr*noise_var))
    c_2 = (1/noise_var)

    constraints = []
    for k in range(K):
        Imask = np.eye(K)
        Imask[k,k] = 0
        constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*(W @ Imask).H @ H[:,k], np.ones(1))), 2)]

    # for k in range(K):
    #     constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*W.H @ H[:,k], np.ones(1))), 2)]
        
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    if prob.status in ['infeasible', 'unbounded']:
        print('infeasible solution')
        return None, None, np.inf

    return W.value, np.linalg.norm(W.value, 'fro')**2

def sdp_omar(H, lmbda, u, M=1000, noise_var=1, min_snr=1):
    """
    Solves the relaxed formulation of Omar et al 2013
    """
    # print('z mask: {},\n z value: {}'.format(z_mask, z_sol))
    N, K = H.shape
    W = cp.Variable((N,K), complex=True)

    obj = cp.Minimize(cp.square(cp.norm(W, 'fro')) + lmbda*(u.T @ cp.norm_inf(W, axis=1)))

    zero = np.zeros(N)
    one = np.ones(N)
    c_1 = (1/np.sqrt(min_snr*noise_var))
    c_2 = (1/noise_var)

    constraints = []
    for k in range(K):
        Imask = np.eye(K)
        Imask[k,k] = 0
        constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*(W @ Imask).H @ H[:,k], np.ones(1))), 2)]

    # for k in range(K):
    #     constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*W.H @ H[:,k], np.ones(1))), 2)]
        
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    if prob.status in ['infeasible', 'unbounded']:
        print('infeasible solution')
        return None, np.inf

    return W.value, np.linalg.norm(W.value, 'fro')**2

if __name__=='__main__':
    N, K = 12,6
    max_ant = 5
    H = np.random.randn(N, K) + 1j*np.random.randn(N, K)
    as_omar(H, max_ant=max_ant)
    # W, obj = sdp_omar(H, 50, np.ones((N,1)))
    # print(np.linalg.norm(W, axis=1), obj)



num selected antennas 5
0.7135416767876461


## Check the time consumption and ogap on different problem sizes 

In [24]:
from as_bb_test import solve_bb
from as_omar import *
from multiprocessing import Pool
import time
import pickle

def solve_bb_pool(arguments):
    instance, max_ant = arguments
    try:
        result = solve_bb(instance, max_ant=max_ant)
    except:
        result = None, None, None, None
    return result

N = [8,12,16]
M = [3,8,12]
L = [4,6,10,12]

combinations = []
num_egs = 30
for n in N:
    for m in M:
        if m <= n:
            for l in L:
                if l<n:
                    combinations.append((n,m,l))
                    
# Compute the ogap of Omar's method using the combinations

save_data_filepath = 'data/omar_eval/omar_snr_evaluation_data.pkl'
save_result_filepath = 'data/omar_eval/omar_snr_evaluation_result.pkl'

result = {'size':[], 'ogap':[], 'time':[], 'sol_rate':[]}
# data = {'instance': [], 'z_opt':[], 'power_opt':[], 'steps':[], 'time':[]}
data = {'size': [], 'data': []}

n,m,l = (8,8,6)
simgas = (1,5,10,50)
np.random.seed(100)

for sigma in sigmas:
# for (n,m,l) in combinations:
    ogap = 0
    time_avg = 0
    solved_instances = 0
    
    instances = np.random.randn(num_egs, 2, n, m)
    arguments_oracle = list(zip(list(instances), [l]*num_egs))

    with Pool(num_egs) as p:
        out_oracle = p.map(solve_bb_pool, arguments_oracle)
        print('pool ended')
    
    optimal_solution_list = [out_oracle[i][0] for i in range(len(out_oracle))]
    optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]
    oracle_time = [out_oracle[i][3] for i in range(len(out_oracle))]
    
    solution_data = (instances, optimal_solution_list, optimal_objective_list, oracle_time)
    data['size'].append((n,m,l))
    data['data'].append(solution_data)
    
    omar_solved_instances = 0
    for i in range(num_egs):
        print("({}, {}, {}), eg: {}".format(n,m,l, i))
        H = instances[i,0,::] + 1j*instances[i,1,::]
        
        if optimal_objective_list[i] is not None:
            # run omar's method
            t1 = time.time()
            power, z = as_omar(H, max_ant=l)
            
            if power is not None:
                ogap += (power-optimal_objective_list[i])/optimal_objective_list[i] * 100
                omar_solved_instances += 1
                
            time_avg += time.time() - t1        
            solved_instances += 1
    
    ogap = ogap/omar_solved_instances 
    omar_solution_rate = omar_solved_instances/solved_instances
    time_avg = time_avg/solved_instances
    
    result['size'].append((n,m,l))
    result['ogap'].append(ogap)
    result['time'].append(time_avg)
    result['sol_rate'].append(omar_solution_rate)
    
    with open(save_result_filepath, 'wb') as handle:
        pickle.dump(data, handle)
    
    with open(save_data_filepath, 'wb') as handle:
        pickle.dump(data, handle)
    
    
        

/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:
/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


timestep timestep

/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


 00 timestepinf  inf 0 0.2227781357623145
 0.26189750897424463inf
 0.16708681992716407


/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:
/scratch/sagar/Projects/combopt/branch-and-bound-ml/antenna_selection/as_bb_test.py:436: RuntimeWarning: invalid value encountered in double_scalars
  if (self.global_U - self.global_L)/abs(self.global_U) < self.epsilon:


...

timestep  0.3267473415028838722  0.219612056890598930.32551222813150504
 0.24223371551532208
timestep 22 0.3740718995555312
 0.2521592266591501timestep 23 0.44103984949409963 0.29457118547055794
timestep 22 timestep0.8400256505990925  220.4520546302035461 
0.36955707292749096 0.2509718169720912
timestep 23 0.3483768279649418 0.2109730125276482
timestep 23 0.3076165628371851 0.21744874516957838
timestep 23 0.4189147588275784 0.24479108162310056
timestep timestep23  230.5647516908444608timestep   0.5794729727752890.3216631174715657323
  0.506676279972891 0.33128863929380040.3833064217259226

timestep 23 0.30300623940192767 0.21931688086371018
timestep 23 0.4795194152834991 0.29895522314987577
timestep 23 0.7385355818867873 0.4668061855397471
timestep 23 0.3674597608049853 0.23768895590359823
timestep 24 timestep0.5555021990625998  0.3128276713746353523
 0.4699929232166574 0.3096515884241185
timestep 24 0.48912222736262917 0.36500931104787043timestep
 23 0.4606520076161944timesteptimestep

timesteptimestep  3333  0.40939988780599984 0.307891179336897750.31812213339611656 
0.2552414740880083
timestep timestep34  330.42844702888993125 timestep 0.325512228131505040.35146084581246595  
0.262361102979830133
 0.3255811944970982 0.27501334308030106
timestep 33 0.3317102224945706 0.27131266161343087
timestep 34 0.3923386644544283 0.33600786976551317
timestep 33 0.5040469194092657 0.4040012957014509
timesteptimestep  33timestep 330.6196568866286049  timestep 33timestep timestep0.48602604767094765timestep 0.2409828408239311233
timesteptimestep0.4988246222039336    340.20995316367193828   0.43412422956785546 33 0.40504911242318037
3333
 33  0.32674734150288387   0.379264846525867670.349865842606729540.397762366814443660.40051030668918060.6347703527505989 0.23868096936794547
   0.31382020703676566
0.49252512806059320.337787538008891330.32464785135560986



timestep 35 0.4694468698588211 0.3459229246186024timestep
 34 0.3784056093697887 timestep 34timesteptimestep0.30104669246252347 

timestep 44 0.31111033547845374 0.28573550966246675
timesteptimestep 44 0.292614896324328 timestep 440.2475000042872822 
0.33071271020623066  0.2733127614701144644
timestep  440.31538955775582633 0.28938992761244625  0.276422056595076570.24899115644167827

timestep 44 0.40504911242318037 0.3769703352910617
timestep 45timestep timestep 0.392338664454428344timesteptimestep   44  0.43010722024818837 440.350938666715812360.3843800951226231 43 0.37817745884291637 
 0.32558119449709820.3378787650735043

 0.5555241126587109 0.28752223124373083
0.5131103210815021
timestep 45 0.301403510342427 0.2581446365839179
timesteptimestep 44  440.4005103066891806timestep  0.3623798142131292
44 0.4701135063401648  0.4352183658395871
0.4131875533428846 0.3840859635374865
timestep timestep44  45 0.487648330274155 0.307891179336897750.4537267160169991
 0.2782648535093276
timestep 44 0.4988246222039336 0.4561274070235149
timestep 44 0.5913352505685473 timestep 0.523534792169995744
 0.4302407263466291 0.398422

timesteptimestep  55 56 0.41318755334288460.31111033547845374  0.39978756645207125
0.30695968537733254
timestep 57 0.33071271020623066 timestep0.3034845747593925timestep 
57  560.31538955775582633  0.32558119449709820.31417275064245875 
0.31314126151784977
timestep 57 0.3843800951226231 0.38148776580274024
timestep timestep56  0.4301072202481883757  0.40442862840389220.3784056093697887
 0.34353197723118095
timestep 57 timestep0.301403510342427  560.27726404675395333 
0.292614896324328 0.27806811938985665
timestep timestep 5658timesteptimestep    0.49882462220393360.285651638149482357 0.2845307777532037
 56  0.284535272133729370.4701135063401648  0.26680273526852780.4610934824081761
0.49543800983018643

timestep 57 0.28938992761244625 0.2859421878987255timestep
 57 0.5913352505685473 0.57934602240947
ended
timestep ended59
 0.41919095876176593timestep  0.4160265747923390757
 0.3255811944970982 0.31924130009536467
timestep 56 0.4131875533428846 0.3997875074293636
timestep 57 0.3111103354

SolverError: Solver 'MOSEK' failed. Try another solver, or solve with verbose=True for more information.

In [50]:
from as_bb_test import solve_bb
from as_omar import *
from multiprocessing import Pool
import time
import pickle
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


def solve_bb_pool(arguments):
    instance, max_ant = arguments
    try:
        result = solve_bb(instance, max_ant=max_ant)
    except:
        result = None, None, None, None
    return result

N = [8,12,16]
M = [3,8,12]
L = [4,6,10,12]

combinations = []
num_egs = 30
for n in N:
    for m in M:
        if m <= n:
            for l in L:
                if l<n:
                    combinations.append((n,m,l))
                    

save_data_filepath = 'data/omar_eval/omar_evaluation_data.pkl'
save_result_filepath = 'data/omar_eval/omar_evaluation_result.pkl'

with open(save_data_filepath, 'rb') as handle:
    data = pickle.load(handle)
    
result = {'size':[], 'ogap':[], 'time':[], 'sol_rate':[]}
# data = {'instance': [], 'z_opt':[], 'power_opt':[], 'steps':[], 'time':[]}

# np.random.seed(100)
# for (n,m,l) in combinations:
if True:
    (n,m,l) = combinations[-3]
    ogap = 0
    time_avg = 0
    solved_instances = 0
    
#     instances = np.random.randn(num_egs, 2, n, m)
    
    omar_solved_instances = 0
    power_tuple = []
    for i in range(num_egs):
        print("({}, {}, {}), eg: {}".format(n,m,l, i))
        H = data['data'][-3][0][i][0,::] + 1j*data['data'][-3][0][i][1,::]
        if data['data'][-3][2][i] is not None:
            # run omar's method
            t1 = time.time()
            power, z = as_omar(H, max_ant=l)
            
            if power is not None:
                ogap += (power-data['data'][-3][2][i])/data['data'][-3][2][i] * 100
                print('power: {}, optimal: {}'.format(power, data['data'][-3][2][i]))
                omar_solved_instances += 1
                power_tuple.append(power, data['data'][-3][2][i])
                
            time_avg += time.time() - t1        
            solved_instances += 1
    
    ogap = ogap/omar_solved_instances 
    omar_solution_rate = omar_solved_instances/solved_instances
    time_avg = time_avg/solved_instances
    
    result['size'].append((n,m,l))
    result['ogap'].append(ogap)
    result['time'].append(time_avg)
    result['sol_rate'].append(omar_solution_rate)
    

        

(16, 12, 6), eg: 0
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
sparse iteration  7
sparse iteration  8
sparse iteration  9
sparse iteration  10
sparse iteration  11
sparse iteration  12
sparse iteration  13
sparse iteration  14
sparse iteration  15
sparse iteration  16
sparse iteration  17
sparse iteration  18
sparse iteration  19
sparse iteration  20
sparse iteration  21
sparse iteration  22
sparse iteration  23
sparse iteration  24
sparse iteration  25
exiting here
num selected antennas 6
5.274103438719228
Before lambda iteration: 6
After lambda iteration: 6
power: 5.274103438719228, optimal: 5.253508696288364
(16, 12, 6), eg: 1
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
sparse iteration  7
sparse iteration  8
exiting here
num selected antennas 6
4.47427136720469
Before lambda iteration: 6
Afte

sparse iteration  12
sparse iteration  13
sparse iteration  14
sparse iteration  15
sparse iteration  16
sparse iteration  17
sparse iteration  18
sparse iteration  19
sparse iteration  20
sparse iteration  21
sparse iteration  22
sparse iteration  23
sparse iteration  24
sparse iteration  25
sparse iteration  26
sparse iteration  27
sparse iteration  28
sparse iteration  29
(16, 12, 6), eg: 17
sparse iteration  0
sparse iteration  1
sparse iteration  2
sparse iteration  3
sparse iteration  4
sparse iteration  5
sparse iteration  6
sparse iteration  7
sparse iteration  8
sparse iteration  9
sparse iteration  10
sparse iteration  11
sparse iteration  12
sparse iteration  13
sparse iteration  14
sparse iteration  15
sparse iteration  16
sparse iteration  17
sparse iteration  18
sparse iteration  19
sparse iteration  20
sparse iteration  21
sparse iteration  22
sparse iteration  23
sparse iteration  24
sparse iteration  25
sparse iteration  26
sparse iteration  27
sparse iteration  28
spa

# Experiment for different SNR

In [ ]:
from as_bb_test import solve_bb
from as_omar import *
from multiprocessing import Pool
import time
import pickle
import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__


def solve_bb_pool(arguments):
    instance, max_ant = arguments
    try:
        result = solve_bb(instance, max_ant=max_ant)
    except:
        result = None, None, None, None
    return result

           
save_data_filepath = 'data/omar_eval/omar_evaluation_data.pkl'
save_result_filepath = 'data/omar_eval/omar_evaluation_result.pkl'

with open(save_data_filepath, 'rb') as handle:
    data = pickle.load(handle)
    
result = {'size':[], 'ogap':[], 'time':[], 'sol_rate':[]}
# data = {'instance': [], 'z_opt':[], 'power_opt':[], 'steps':[], 'time':[]}


n,m,l = (8,8,6)
simgas = (1,5,10,50)
# np.random.seed(100)
for sigma in sigmas:
# if True:
    ogap = 0
    time_avg = 0
    solved_instances = 0
    
#     instances = np.random.randn(num_egs, 2, n, m)
    
    omar_solved_instances = 0
    power_tuple = []
    for i in range(num_egs):
        print("({}, {}, {}), eg: {}".format(n,m,l, i))
        H = data['data'][-3][0][i][0,::] + 1j*data['data'][-3][0][i][1,::]
        if data['data'][-3][2][i] is not None:
            # run omar's method
            t1 = time.time()
            power, z = as_omar(H, max_ant=l)
            
            if power is not None:
                ogap += (power-data['data'][-3][2][i])/data['data'][-3][2][i] * 100
                print('power: {}, optimal: {}'.format(power, data['data'][-3][2][i]))
                omar_solved_instances += 1
                power_tuple.append(power, data['data'][-3][2][i])
                
            time_avg += time.time() - t1        
            solved_instances += 1
    
    ogap = ogap/omar_solved_instances 
    omar_solution_rate = omar_solved_instances/solved_instances
    time_avg = time_avg/solved_instances
    
    result['size'].append((n,m,l))
    result['ogap'].append(ogap)
    result['time'].append(time_avg)
    result['sol_rate'].append(omar_solution_rate)
    

        

In [57]:
# check example 29
H = data['data'][-3][0][i][0,::] + 1j*data['data'][-3][0][i][1,::]

result = solve_bb(data['data'][-3][0][i], max_ant=l, max_iter = 10000)


timestep 0 inf 0.6932142809009355
timestep 1 inf 0.6932151145132005
timestep 2 inf 0.6932140165628387
timestep 3 inf 0.6932149342898318
timestep 4 inf 0.693214706026119
timestep 5 inf 0.693214753109464
timestep 6 66.19771762150376 0.7389190484851248
timestep 7 24.59085974566502 0.7619753178022851
timestep 8 24.59085974566502 0.7619752253138317
timestep 9 24.59085974566502 0.7672024117181877
timestep 10 24.59085974566502 0.7672029437784418
timestep 11 24.59085974566502 0.7672024938811525
timestep 12 24.59085974566502 0.7672023498009475
timestep 13 24.59085974566502 0.7672024241622174
timestep 14 24.59085974566502 0.7672033761748466
timestep 15 24.59085974566502 0.769445254535254
timestep 16 24.59085974566502 0.7694455719771782
timestep 17 24.59085974566502 0.769445219340923
timestep 18 24.59085974566502 0.7694456132327503
timestep 19 24.59085974566502 0.770750828667492
timestep 20 24.59085974566502 0.7707521218532863
timestep 21 24.59085974566502 0.7707516001554152
timestep 22 24.590859

timestep 167 10.014213376111375 1.0475591762762224
timestep 168 10.014213376111375 1.050059260680173
timestep 169 10.014213376111375 1.050182434548039
timestep 170 10.014213376111375 1.0504939768149153
timestep 171 10.014213376111375 1.0504937984226312
timestep 172 10.014213376111375 1.0511344612513134
timestep 173 10.014213376111375 1.0511341020277223
timestep 174 10.014213376111375 1.0517071627270347
timestep 175 10.014213376111375 1.0517081203524274
timestep 176 10.014213376111375 1.0573829229216605
timestep 177 10.014213376111375 1.0573825619586055
timestep 178 10.014213376111375 1.0573816058320658
timestep 179 10.014213376111375 1.0579454752495776
timestep 180 10.014213376111375 1.058019497717119
timestep 181 10.014213376111375 1.060112861189407
timestep 182 10.014213376111375 1.060113550811846
timestep 183 10.014213376111375 1.0604454186275447
timestep 184 10.014213376111375 1.061036789936125
timestep 185 10.014213376111375 1.061035969309388
timestep 186 10.014213376111375 1.0610

timestep 331 8.438026842718964 1.1940160120622958
timestep 332 8.438026842718964 1.1942929028438223
timestep 333 6.081465229840527 1.1948077084954376
timestep 334 6.081465229840527 1.1949226094279906
timestep 335 6.081465229840527 1.1959346858431437
timestep 336 6.081465229840527 1.195935366978762
timestep 337 6.081465229840527 1.1981072866828357
timestep 338 6.081465229840527 1.1991118283243798
timestep 339 6.081465229840527 1.1996239124536436
timestep 340 6.081465229840527 1.2003509089528674
timestep 341 6.081465229840527 1.2003506944358875
timestep 342 6.081465229840527 1.2061432818980409
timestep 343 6.081465229840527 1.206143335206464
timestep 344 6.081465229840527 1.2061425570033393
timestep 345 6.081465229840527 1.206143108011494
timestep 346 6.081465229840527 1.2072619671981208
timestep 347 6.081465229840527 1.2074148693833837
timestep 348 6.081465229840527 1.2074142482274508
timestep 349 6.081465229840527 1.2074141070521656
timestep 350 6.081465229840527 1.2094377851370666
tim

timestep 496 6.081465229840527 1.2988702820620082
timestep 497 6.081465229840527 1.298869359695393
timestep 498 6.081465229840527 1.2989556684551944
timestep 499 6.081465229840527 1.2989565103240972
timestep 500 6.081465229840527 1.2998914518424074
timestep 501 6.081465229840527 1.3012109677758035
timestep 502 6.081465229840527 1.3018165804947797
timestep 503 6.081465229840527 1.301816550763318
timestep 504 6.081465229840527 1.3023381797728113
timestep 505 6.081465229840527 1.302794636716883
timestep 506 6.081465229840527 1.3029775475305598
timestep 507 6.081465229840527 1.3029775470320804
timestep 508 6.081465229840527 1.3036564042603933
timestep 509 6.081465229840527 1.3059586465053497
timestep 510 6.081465229840527 1.3059598276381061
timestep 511 6.081465229840527 1.3059586107845307
timestep 512 6.081465229840527 1.3063669376280915
timestep 513 6.081465229840527 1.3068302697127068
timestep 514 6.081465229840527 1.3068300715846535
timestep 515 6.081465229840527 1.3068293691965112
tim

timestep 661 6.081465229840527 1.415791957933798
timestep 662 6.081465229840527 1.415792634611643
timestep 663 6.081465229840527 1.420525931011092
timestep 664 6.081465229840527 1.4205265495320956
timestep 665 6.081465229840527 1.4205263451105667
timestep 666 6.081465229840527 1.4228412658523333
timestep 667 6.081465229840527 1.423297534289047
timestep 668 6.081465229840527 1.4246006316772146
timestep 669 6.081465229840527 1.4270362327381116
timestep 670 6.081465229840527 1.4291827602828815
timestep 671 6.081465229840527 1.42918251759413
timestep 672 6.081465229840527 1.4298376276435578
timestep 673 6.081465229840527 1.4303107286611958
timestep 674 6.081465229840527 1.4303119450675983
timestep 675 6.081465229840527 1.430310350686461
timestep 676 6.081465229840527 1.4303115385468168
timestep 677 6.081465229840527 1.4303110476736927
timestep 678 6.081465229840527 1.4310399774239133
timestep 679 6.081465229840527 1.4310395234319373
timestep 680 6.081465229840527 1.4310398700514266
timeste

...

timestep 3590 4.317657557273796 3.470817199241954
timestep 3591 4.317657557273796 3.470816516677474
timestep 3592 4.317657557273796 3.4718863813288148
timestep 3593 4.317657557273796 3.472078858267738
timestep 3594 4.317657557273796 3.4730666230729814
timestep 3595 4.317657557273796 3.475193280606713
timestep 3596 4.317657557273796 3.4753110924973876
timestep 3597 4.317657557273796 3.475411857350373
timestep 3598 4.317657557273796 3.476548429767339
timestep 3599 4.317657557273796 3.4765474371446756
timestep 3600 4.317657557273796 3.476547641631148
timestep 3601 4.317657557273796 3.47825247632483
timestep 3602 4.317657557273796 3.4811492530923447
timestep 3603 4.317657557273796 3.4811498169952206
timestep 3604 4.317657557273796 3.4818614061396653
timestep 3605 4.317657557273796 3.483994026980868
timestep 3606 4.317657557273796 3.4848514079635575
timestep 3607 4.317657557273796 3.484850680917635
timestep 3608 4.317657557273796 3.484861728952413
timestep 3609 4.317657557273796 3.485025939

timestep 3750 4.317657557273796 3.6312073776863483
timestep 3751 4.317657557273796 3.6314677212419118
timestep 3752 4.317657557273796 3.631467853780212
timestep 3753 4.317657557273796 3.6330672766952072
timestep 3754 4.317657557273796 3.6330664858746298
timestep 3755 4.317657557273796 3.635333406120457
timestep 3756 4.317657557273796 3.6365327084021826
timestep 3757 4.317657557273796 3.6365323950072512
timestep 3758 4.317657557273796 3.6365312769200284
timestep 3759 4.317657557273796 3.6393686253735615
timestep 3760 4.317657557273796 3.6393685881139946
timestep 3761 4.317657557273796 3.6393703870975562
timestep 3762 4.317657557273796 3.6393708241552933
timestep 3763 4.317657557273796 3.63939579502418
timestep 3764 4.317657557273796 3.640493379701826
timestep 3765 4.317657557273796 3.640492867074051
timestep 3766 4.317657557273796 3.640494067677134
timestep 3767 4.317657557273796 3.642931489839699
timestep 3768 4.317657557273796 3.6456066353146706
timestep 3769 4.317657557273796 3.64598

timestep 3912 4.317657557273796 3.793790555718019
timestep 3913 4.317657557273796 3.7955185898537778
timestep 3914 4.317657557273796 3.7955195388188825
timestep 3915 4.317657557273796 3.795935764983037
timestep 3916 4.317657557273796 3.800910903856874
timestep 3917 4.317657557273796 3.800911125150879
timestep 3918 4.317657557273796 3.80200752395912
timestep 3919 4.317657557273796 3.8024573674329885
timestep 3920 4.317657557273796 3.8028898859268274
timestep 3921 4.317657557273796 3.8041251151133193
timestep 3922 4.317657557273796 3.80432262535759
timestep 3923 4.317657557273796 3.8060727765218867
timestep 3924 4.317657557273796 3.8074685636985786
timestep 3925 4.317657557273796 3.807467250481792
timestep 3926 4.317657557273796 3.8075990517058953
timestep 3927 4.317657557273796 3.808449152550989
timestep 3928 4.317657557273796 3.8084474004467337
timestep 3929 4.317657557273796 3.8097947212363272
timestep 3930 4.317657557273796 3.809794996720835
timestep 3931 4.317657557273796 3.81413277

timestep 4073 4.317657557273796 3.952250281307837
timestep 4074 4.317657557273796 3.9522498516901186
timestep 4075 4.317657557273796 3.9529260766132586
timestep 4076 4.317657557273796 3.9529252471871374
timestep 4077 4.317657557273796 3.9529650421327776
timestep 4078 4.317657557273796 3.9556757651481926
timestep 4079 4.317657557273796 3.955911383771184
timestep 4080 4.317657557273796 3.955910086610538
timestep 4081 4.317657557273796 3.956277432360142
infeasible solution
timestep 4082 4.317657557273796 3.956493651309606
timestep 4083 4.317657557273796 3.957196115793075
timestep 4084 4.317657557273796 3.957196637803794
timestep 4085 4.317657557273796 3.957196453538427
timestep 4086 4.317657557273796 3.9571955013263533
timestep 4087 4.317657557273796 3.9571975668219044
timestep 4088 4.317657557273796 3.958471137269151
timestep 4089 4.317657557273796 3.9586844225201974
timestep 4090 4.317657557273796 3.9591452174880977
timestep 4091 4.317657557273796 3.961161364132671
timestep 4092 4.31765

timestep 4234 4.317657557273796 4.112550653259075
timestep 4235 4.317657557273796 4.112711816868478
timestep 4236 4.317657557273796 4.115854960711199
timestep 4237 4.317657557273796 4.116843779947681
timestep 4238 4.317657557273796 4.121876371254364
timestep 4239 4.317657557273796 4.123979951148041
timestep 4240 4.317657557273796 4.124407906766758
timestep 4241 4.317657557273796 4.125994814234839
infeasible solution
timestep 4242 4.317657557273796 4.127360482831571
timestep 4243 4.317657557273796 4.127362082876349
timestep 4244 4.317657557273796 4.128532149014405
timestep 4245 4.317657557273796 4.132339512409002
timestep 4246 4.317657557273796 4.137432728900553
timestep 4247 4.317657557273796 4.137431949414866
timestep 4248 4.317657557273796 4.137749391386907
timestep 4249 4.317657557273796 4.139723609351787
timestep 4250 4.317657557273796 4.143628104626379
timestep 4251 4.317657557273796 4.1436289472221395
timestep 4252 4.317657557273796 4.145055191545075
timestep 4253 4.3176575572737

In [51]:
data['data'][-3][1][29]

array([0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 1., 0., 1., 0., 0.])

In [53]:
data['data'][-3][2][29]

5.431862622454325

In [54]:
z

array([0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0])

In [41]:
for (n,m,l) in combinations:
    H = np.random.randn(n,m) + 1j*np.random.randn(n,m)
    power, z = as_omar(H, max_ant=l)

sparse iteration  0
sparse iteration  1
exiting here
iteration 1 500000.0 3 0 1000000.0
iteration 2 250000.0 3 0 500000.0
iteration 3 125000.0 3 0 250000.0
iteration 4 62500.0 3 0 125000.0
iteration 5 31250.0 3 0 62500.0
iteration 6 15625.0 3 0 31250.0
iteration 7 7812.5 3 0 15625.0
iteration 8 3906.25 3 0 7812.5
iteration 9 1953.125 3 0 3906.25
iteration 10 976.5625 3 0 1953.125
iteration 11 488.28125 3 0 976.5625
iteration 12 244.140625 3 0 488.28125
iteration 13 122.0703125 3 0 244.140625
iteration 14 61.03515625 3 0 122.0703125
iteration 15 30.517578125 3 0 61.03515625
iteration 16 15.2587890625 4 0 30.517578125
num selected antennas 4
0.34676417017802247
Before lambda iteration: 3
After lambda iteration: 3
sparse iteration  0
exiting here
iteration 1 500000.0 3 0 1000000.0
iteration 2 250000.0 3 0 500000.0
iteration 3 125000.0 3 0 250000.0
iteration 4 62500.0 3 0 125000.0
iteration 5 31250.0 3 0 62500.0
iteration 6 15625.0 3 0 31250.0
iteration 7 7812.5 3 0 15625.0
iteration 8 390

iteration 15 30.517578125 7 0 61.03515625
iteration 16 15.2587890625 7 0 30.517578125
iteration 17 7.62939453125 7 0 15.2587890625
iteration 18 3.814697265625 7 0 7.62939453125
iteration 19 1.9073486328125 8 0 3.814697265625
iteration 20 0.95367431640625 9 0 1.9073486328125
iteration 21 0.476837158203125 10 0 0.95367431640625
num selected antennas 10
1.3754382017711648
Before lambda iteration: 7
After lambda iteration: 7
sparse iteration  0
exiting here
num selected antennas 4
0.2671700971626158
Before lambda iteration: 4
After lambda iteration: 4
sparse iteration  0
exiting here
iteration 1 500000.0 5 0 1000000.0
iteration 2 250000.0 5 0 500000.0
iteration 3 125000.0 5 0 250000.0
iteration 4 62500.0 5 0 125000.0
iteration 5 31250.0 5 0 62500.0
iteration 6 15625.0 5 0 31250.0
iteration 7 7812.5 5 0 15625.0
iteration 8 3906.25 5 0 7812.5
iteration 9 1953.125 5 0 3906.25
iteration 10 976.5625 5 0 1953.125
iteration 11 488.28125 5 0 976.5625
iteration 12 244.140625 5 0 488.28125
iteration

iteration 15 30.517578125 9 0 61.03515625
iteration 16 15.2587890625 9 0 30.517578125
iteration 17 7.62939453125 9 0 15.2587890625
iteration 18 3.814697265625 9 0 7.62939453125
iteration 19 1.9073486328125 9 0 3.814697265625
iteration 20 0.95367431640625 10 0 1.9073486328125
num selected antennas 10
1.062852777675884
Before lambda iteration: 9
After lambda iteration: 9
sparse iteration  0
sparse iteration  1
sparse iteration  2
exiting here
iteration 1 500000.0 11 0 1000000.0
iteration 2 250000.0 11 0 500000.0
iteration 3 125000.0 11 0 250000.0
iteration 4 62500.0 11 0 125000.0
iteration 5 31250.0 11 0 62500.0
iteration 6 15625.0 11 0 31250.0
iteration 7 7812.5 11 0 15625.0
iteration 8 3906.25 11 0 7812.5
iteration 9 1953.125 11 0 3906.25
iteration 10 976.5625 11 0 1953.125
iteration 11 488.28125 11 0 976.5625
iteration 12 244.140625 11 0 488.28125
iteration 13 122.0703125 11 0 244.140625
iteration 14 61.03515625 11 0 122.0703125
iteration 15 30.517578125 11 0 61.03515625
iteration 16 

In [19]:
N = [8,12,16]
M = [3,8,12]
L = [4,6,10,12]

combinations = []
num_egs = 30
for n in N:
    for m in M:
        if m <= n:
            for l in L:
                if l<n:
                    combinations.append((n,m,l))
for (n,m,l) in combinations:
    print(n,m,l)

8 3 4
8 3 6
8 8 4
8 8 6
12 3 4
12 3 6
12 3 10
12 8 4
12 8 6
12 8 10
12 12 4
12 12 6
12 12 10
16 3 4
16 3 6
16 3 10
16 3 12
16 8 4
16 8 6
16 8 10
16 8 12
16 12 4
16 12 6
16 12 10
16 12 12


In [20]:
len(combinations)

25

In [14]:
data['data'][3]

(array([[[[ 2.88402662e-01, -1.56450512e+00, -8.75314883e-01, ...,
            1.27102966e-01,  8.09550606e-01,  1.89195904e+00],
          [ 8.73103818e-03,  2.65289976e-01, -6.33359502e-01, ...,
           -1.76693206e+00,  4.30382398e-01, -4.61754277e-01],
          [ 1.39992114e+00, -2.03886402e+00, -5.85818543e-01, ...,
            3.61280169e-01,  4.78882691e-01, -4.00604520e-01],
          ...,
          [-3.60011209e-01, -1.03284149e+00, -8.41385892e-02, ...,
            4.69219673e-01,  1.09010770e+00, -2.15513935e+00],
          [-2.18135934e+00, -1.18651515e-01, -3.98647027e-02, ...,
           -6.09701116e-01,  4.87821711e-01,  8.73617285e-02],
          [-4.35888118e-01, -8.10160198e-02, -6.30346689e-01, ...,
           -1.25412389e+00,  8.17812471e-01, -2.97381866e-01]],
 
         [[ 1.64883679e+00,  3.14171391e-01,  8.42641227e-01, ...,
            2.73992422e-01,  7.63971619e-01,  6.67901276e-01],
          [-5.90452030e-01, -5.15673101e-01, -7.44329688e-01, ...,
     

In [ ]:
N = [8,12,16]
M = [3,8,12]
L = [4,6,10,12]

combinations = []
num_egs = 30
for n in N:
    for m in N:
        if m <= n:
            for l in L:
                if l<n:
                    combinations.append((n,m,l))

In [2]:
combinations

[(8, 8, 4),
 (8, 8, 6),
 (12, 8, 4),
 (12, 8, 6),
 (12, 8, 10),
 (12, 12, 4),
 (12, 12, 6),
 (12, 12, 10),
 (16, 8, 4),
 (16, 8, 6),
 (16, 8, 10),
 (16, 8, 12),
 (16, 12, 4),
 (16, 12, 6),
 (16, 12, 10),
 (16, 12, 12),
 (16, 16, 4),
 (16, 16, 6),
 (16, 16, 10),
 (16, 16, 12)]

In [12]:
import pickle

with open(save_result_filepath, 'wb') as handle:
    pickle.dump(result, handle)
    
with open(save_data_filepath, 'wb') as handle:
    pickle.dump(data, handle)
    

In [42]:
import pickle
save_data_filepath = 'data/omar_eval/omar_evaluation_data.pkl'
save_result_filepath = 'data/omar_eval/omar_evaluation_result.pkl'

with open(save_data_filepath, 'rb') as handle:
    data = pickle.load(handle)
    
with open(save_result_filepath, 'rb') as handle:
    result = pickle.load(handle)

In [12]:
data['data'][0][]

[0.4005103066891806,
 0.39776236681444366,
 0.33071271020623066,
 0.3784056093697887,
 0.3255811944970982,
 0.28453527213372937,
 0.28938992761244625,
 0.30789117933689775,
 0.3923386644544283,
 0.43010722024818837,
 0.301403510342427,
 0.3983396906727539,
 0.31111033547845374,
 0.40504911242318037,
 0.4701135063401648,
 0.4988246222039336,
 0.5555241126587109,
 0.4302407263466291,
 0.37926484652586767,
 0.2856516381494823,
 0.487648330274155,
 0.4131875533428846,
 0.4358372309052946,
 0.292614896324328,
 0.5913352505685473,
 0.24098284082393112,
 0.41919095876176593,
 0.31538955775582633,
 0.3843800951226231,
 0.3317102224945706]

In [15]:
result.keys()

dict_keys(['size', 'ogap', 'time', 'sol_rate'])

In [16]:
result['size']

[(8, 3, 4),
 (8, 3, 6),
 (8, 8, 4),
 (8, 8, 6),
 (12, 3, 4),
 (12, 3, 6),
 (12, 3, 10),
 (12, 8, 4),
 (12, 8, 6),
 (12, 8, 10),
 (12, 12, 4),
 (12, 12, 6),
 (12, 12, 10),
 (16, 3, 4),
 (16, 3, 6),
 (16, 3, 10),
 (16, 3, 12),
 (16, 8, 4),
 (16, 8, 6),
 (16, 8, 10),
 (16, 8, 12),
 (16, 12, 4),
 (16, 12, 6),
 (16, 12, 10),
 (16, 12, 12)]

In [ ]:
Try for (12, 8, 4) and (8,8,4)

In [20]:
for i in range(len(result['size'])):
    print('size: {}, ogap: {}, time: {}, sol rate: {}'.format(result['size'][i], result['ogap'][i], result['time'][i], result['sol_rate'][i]))
    

size: (8, 3, 4), ogap: 2.8512433097382224, time: 0.7785226583480835, sol rate: 1.0
size: (8, 3, 6), ogap: 1.0386193214851176, time: 1.2516060988108317, sol rate: 1.0
size: (8, 8, 4), ogap: 5.053933982420465, time: 3.0280043749973693, sol rate: 0.4482758620689655
size: (8, 8, 6), ogap: 5.441923493258856, time: 1.302332599957784, sol rate: 1.0
size: (12, 3, 4), ogap: 5.3036140418175775, time: 0.983303968111674, sol rate: 1.0
size: (12, 3, 6), ogap: 2.3929738482371707, time: 1.160412589708964, sol rate: 1.0
size: (12, 3, 10), ogap: 0.6459367444374032, time: 1.8711916049321493, sol rate: 1.0
size: (12, 8, 4), ogap: 9.62205530077256, time: 2.615790581703186, sol rate: 0.6333333333333333
size: (12, 8, 6), ogap: 10.073832654678844, time: 1.6217383845099087, sol rate: 1.0
size: (12, 8, 10), ogap: 1.4887403624278086, time: 3.3045375108718873, sol rate: 1.0
size: (12, 12, 4), ogap: 1000000000.0, time: 6.514667446796711, sol rate: 0.0
size: (12, 12, 6), ogap: 2.3555288071355482, time: 5.241326851

In [21]:
data['data'][-3]

(array([[[[-7.46716680e-01,  6.95551338e-01,  3.81039919e-01, ...,
            6.88822033e-01, -1.86978716e+00,  1.31145805e+00],
          [ 1.87624263e+00,  1.55651402e+00,  1.58950475e-01, ...,
           -5.87396760e-01, -1.29948930e+00,  9.19449131e-02],
          [-2.26078974e-01,  9.51603961e-03, -1.10459880e+00, ...,
            8.68894592e-01,  2.88611027e-01, -7.50858123e-01],
          ...,
          [ 6.28677158e-01, -3.61206211e-01, -1.76783194e+00, ...,
           -2.53655489e-01, -5.58115186e-01,  3.80746359e-01],
          [-1.19009020e+00, -8.24310610e-01,  3.21339711e-01, ...,
            1.02443597e+00,  2.39952525e-01, -8.15026995e-01],
          [ 1.11468948e-01,  8.53625752e-01, -1.23694226e+00, ...,
            1.97427680e-01, -9.83171905e-01, -1.76600521e+00]],
 
         [[ 9.20571502e-01, -6.77282330e-01, -4.86360858e-01, ...,
            4.93243298e-01, -9.87016704e-01,  2.86078545e+00],
          [ 2.98260893e-01,  6.19464126e-01, -1.43832345e+00, ...,
     

In [9]:
data.keys()

dict_keys(['size', 'data'])

In [14]:
data['size']

(12, 8, 10)

In [ ]:
']

In [13]:
len(data['data'])

5

In [13]:
with open(save_result_filepath, 'rb') as handle:
    result = pickle.load(handle)

In [15]:
result.keys()

dict_keys(['size', 'data'])

In [39]:
def as_omar(H, max_ant=5):
    lmbda_lb = 0
    lmbda_ub = 1e6
    # global lmbda_lb, lmbda_ub
    N,K = H.shape

    # step 1
    u = np.zeros((N,1))
    u_new = np.ones((N,1))
    r = 0
    max_iter = 30
    # while np.linalg.norm(u-u_new)>0.0001 and r < max_iter:
    while r < max_iter:
        print('sparse iteration  {}'.format(r))
        r += 1
        u = u_new.copy()
        W, _ = sparse_iteration(H, u)
        a = np.linalg.norm(W, axis=1)
        mask = (a>0.001)*1
        if mask.sum()<= max_ant:
            print('exiting here')
            break
        u_new = 1/(np.linalg.norm(W, axis=1) + 1e-5)
    prelim_mask = mask.copy()

    # if mask.sum() > max_ant:
    #     return
    before_iter_ant_count = mask.sum()
    
    if mask.sum() > max_ant:
        return None, mask.copy()
    
    # step 2
    r = 0
    max_iter = 50

    while mask.sum() != max_ant and r < max_iter:
        r += 1
        # if mask.sum() < max_ant:
        lmbda = lmbda_lb + (lmbda_ub - lmbda_lb)/2
#         print(u_new, lmbda)
        W, _ = sdp_omar(H, lmbda, u_new)
#         print(W)
        a = np.linalg.norm(W, axis=1)
        mask = (a>0.001)*1
        print('iteration {}'.format(r), lmbda, mask.sum(), lmbda_lb, lmbda_ub)
        if mask.sum() == max_ant:
            break
        elif mask.sum() > max_ant:
            lmbda_lb = lmbda
        elif mask.sum() < max_ant:
            lmbda_ub = lmbda
    if mask.sum()>max_ant:
        mask = prelim_mask.copy()    
    print('num selected antennas', mask.sum())

    after_iter_ant_count = mask.sum()

    # step 3
    W, obj = solve_beamforming_with_selected_antennas(H, mask)
    print(obj)
    print('Before lambda iteration: {}'.format(before_iter_ant_count))
    print('After lambda iteration: {}'.format(after_iter_ant_count))

    return obj.copy(), mask.copy()

def sparse_iteration(H, u, M=1000, noise_var=1, min_snr=1):
    """
    Solves the relaxed formulation of Omar et al 2013
    """
    # print('z mask: {},\n z value: {}'.format(z_mask, z_sol))
    N, K = H.shape
    W = cp.Variable((N,K), complex=True)

    obj = cp.Minimize((u.T @ cp.norm_inf(W, axis=1)))

    zero = np.zeros(N)
    one = np.ones(N)
    c_1 = (1/np.sqrt(min_snr*noise_var))
    c_2 = (1/noise_var)

    constraints = []
    for k in range(K):
        Imask = np.eye(K)
        Imask[k,k] = 0
        constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*(W @ Imask).H @ H[:,k], np.ones(1))), 2)]

    # for k in range(K):
    #     constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*W.H @ H[:,k], np.ones(1))), 2)]
        
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    if prob.status in ['infeasible', 'unbounded']:
        print('infeasible solution')
        return None, None, np.inf

    return W.value, np.linalg.norm(W.value, 'fro')**2

def sdp_omar(H, lmbda, u, M=1000, noise_var=1, min_snr=1):
    """
    Solves the relaxed formulation of Omar et al 2013
    """
    # print('z mask: {},\n z value: {}'.format(z_mask, z_sol))
    N, K = H.shape
    W = cp.Variable((N,K), complex=True)

    obj = cp.Minimize(cp.square(cp.norm(W, 'fro')) + lmbda*(u.T @ cp.norm_inf(W, axis=1)))

    zero = np.zeros(N)
    one = np.ones(N)
    c_1 = (1/np.sqrt(min_snr*noise_var))
    c_2 = (1/noise_var)

    constraints = []
    for k in range(K):
        Imask = np.eye(K)
        Imask[k,k] = 0
        constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*(W @ Imask).H @ H[:,k], np.ones(1))), 2)]

    # for k in range(K):
    #     constraints += [c_1*cp.real(np.expand_dims(H[:,k], axis=0) @ W[:,k]) >= cp.norm(cp.hstack((c_2*W.H @ H[:,k], np.ones(1))), 2)]
        
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    if prob.status in ['infeasible', 'unbounded']:
        print('infeasible solution')
        return None, np.inf

    return W.value, np.linalg.norm(W.value, 'fro')**2

In [40]:
power, z = as_omar(H, max_ant=l)

sparse iteration  0
sparse iteration  1
exiting here
iteration 1 500000.0 2 0 1000000.0
iteration 2 250000.0 2 0 500000.0
iteration 3 125000.0 2 0 250000.0
iteration 4 62500.0 2 0 125000.0
iteration 5 31250.0 2 0 62500.0
iteration 6 15625.0 2 0 31250.0
iteration 7 7812.5 2 0 15625.0
iteration 8 3906.25 2 0 7812.5
iteration 9 1953.125 2 0 3906.25
iteration 10 976.5625 2 0 1953.125
iteration 11 488.28125 2 0 976.5625
iteration 12 244.140625 2 0 488.28125
iteration 13 122.0703125 2 0 244.140625
iteration 14 61.03515625 2 0 122.0703125
iteration 15 30.517578125 2 0 61.03515625
iteration 16 15.2587890625 2 0 30.517578125
iteration 17 7.62939453125 2 0 15.2587890625
iteration 18 3.814697265625 2 0 7.62939453125
iteration 19 1.9073486328125 2 0 3.814697265625
iteration 20 0.95367431640625 2 0 1.9073486328125
iteration 21 0.476837158203125 2 0 0.95367431640625
iteration 22 0.2384185791015625 3 0 0.476837158203125
iteration 23 0.11920928955078125 5 0 0.2384185791015625
iteration 24 0.1788139343

In [36]:
l

4

In [8]:
n,m,l

(8, 8, 4)

In [16]:
time_avg = 0
import time
for i in range(num_egs):
    print("({}, {}, {}), eg: {}".format(n,m,l, i))
    H = instances[i,0,::] + 1j*instances[i,1,::]

    if optimal_objective_list[i] is not None:
        # run omar's method
        t1 = time.time()
        power, z = as_omar(H, max_ant=l)

        ogap += (power-optimal_objective_list[i])/optimal_objective_list[i] * 100
        time_avg += time.time() - t1

        solved_instances += 1

ogap = ogap/solve_instances
time_avg = time_avg/solved_instances
result['size'].append((n,m,l))
result['ogap'].append(ogap)
result['time'].append(time_avg)
with open(save_result_filepath, 'wb') as handle:
    pickle.dump(data, handle)

with open(save_data_filepath, 'wb') as handle:
    pickle.dump(data, handle)




(8, 8, 4), eg: 0
num selected antennas 4
3.22872634859457
(8, 8, 4), eg: 1
num selected antennas 4
6.639273854549036
(8, 8, 4), eg: 2
iteration 1 50.0000005 6 1e-06 100
iteration 2 75.00000025 6 50.0000005 100
iteration 3 87.500000125 6 75.00000025 100
iteration 4 93.75000006249999 6 87.500000125 100
iteration 5 96.87500003125 6 93.75000006249999 100
iteration 6 98.437500015625 6 96.87500003125 100
iteration 7 99.2187500078125 6 98.437500015625 100
iteration 8 99.60937500390625 6 99.2187500078125 100
iteration 9 99.80468750195313 6 99.60937500390625 100
iteration 10 99.90234375097657 6 99.80468750195313 100
iteration 11 99.95117187548828 6 99.90234375097657 100
iteration 12 99.97558593774414 6 99.95117187548828 100
iteration 13 99.98779296887207 6 99.97558593774414 100
iteration 14 99.99389648443604 6 99.98779296887207 100
iteration 15 99.99694824221802 6 99.99389648443604 100
iteration 16 99.99847412110901 6 99.99694824221802 100
iteration 17 99.9992370605545 6 99.99847412110901 100
i

iteration 49 99.99999999999983 5 99.99999999999964 100
iteration 50 99.99999999999991 5 99.99999999999983 100
num selected antennas 5
3.9589898938180115
(8, 8, 4), eg: 7
iteration 1 50.0000005 5 1e-06 100
iteration 2 75.00000025 5 50.0000005 100
iteration 3 87.500000125 5 75.00000025 100
iteration 4 93.75000006249999 5 87.500000125 100
iteration 5 96.87500003125 5 93.75000006249999 100
iteration 6 98.437500015625 5 96.87500003125 100
iteration 7 99.2187500078125 5 98.437500015625 100
iteration 8 99.60937500390625 5 99.2187500078125 100
iteration 9 99.80468750195313 5 99.60937500390625 100
iteration 10 99.90234375097657 5 99.80468750195313 100
iteration 11 99.95117187548828 5 99.90234375097657 100
iteration 12 99.97558593774414 5 99.95117187548828 100
iteration 13 99.98779296887207 5 99.97558593774414 100
iteration 14 99.99389648443604 5 99.98779296887207 100
iteration 15 99.99694824221802 5 99.99389648443604 100
iteration 16 99.99847412110901 5 99.99694824221802 100
iteration 17 99.999

iteration 48 99.99999999999964 5 99.99999999999929 100
iteration 49 99.99999999999983 5 99.99999999999964 100
iteration 50 99.99999999999991 5 99.99999999999983 100
num selected antennas 5
2.160325962922556
(8, 8, 4), eg: 11
iteration 1 50.0000005 6 1e-06 100
iteration 2 75.00000025 6 50.0000005 100
iteration 3 87.500000125 6 75.00000025 100
iteration 4 93.75000006249999 6 87.500000125 100
iteration 5 96.87500003125 6 93.75000006249999 100
iteration 6 98.437500015625 6 96.87500003125 100
iteration 7 99.2187500078125 6 98.437500015625 100
iteration 8 99.60937500390625 6 99.2187500078125 100
iteration 9 99.80468750195313 6 99.60937500390625 100
iteration 10 99.90234375097657 6 99.80468750195313 100
iteration 11 99.95117187548828 6 99.90234375097657 100
iteration 12 99.97558593774414 6 99.95117187548828 100
iteration 13 99.98779296887207 6 99.97558593774414 100
iteration 14 99.99389648443604 6 99.98779296887207 100
iteration 15 99.99694824221802 6 99.99389648443604 100
iteration 16 99.998

iteration 45 99.99999999999716 6 99.99999999999432 100
iteration 46 99.99999999999858 6 99.99999999999716 100
iteration 47 99.99999999999929 6 99.99999999999858 100
iteration 48 99.99999999999964 6 99.99999999999929 100
iteration 49 99.99999999999983 6 99.99999999999964 100
iteration 50 99.99999999999991 6 99.99999999999983 100
num selected antennas 6
1.456409507948898
(8, 8, 4), eg: 18
num selected antennas 4
5.292132962106845
(8, 8, 4), eg: 19
iteration 1 50.0000005 5 1e-06 100
iteration 2 75.00000025 5 50.0000005 100
iteration 3 87.500000125 5 75.00000025 100
iteration 4 93.75000006249999 5 87.500000125 100
iteration 5 96.87500003125 5 93.75000006249999 100
iteration 6 98.437500015625 5 96.87500003125 100
iteration 7 99.2187500078125 5 98.437500015625 100
iteration 8 99.60937500390625 5 99.2187500078125 100
iteration 9 99.80468750195313 5 99.60937500390625 100
iteration 10 99.90234375097657 5 99.80468750195313 100
iteration 11 99.95117187548828 5 99.90234375097657 100
iteration 12 9

iteration 41 99.99999999995453 5 99.99999999990905 100
iteration 42 99.99999999997726 5 99.99999999995453 100
iteration 43 99.99999999998863 5 99.99999999997726 100
iteration 44 99.99999999999432 5 99.99999999998863 100
iteration 45 99.99999999999716 5 99.99999999999432 100
iteration 46 99.99999999999858 5 99.99999999999716 100
iteration 47 99.99999999999929 5 99.99999999999858 100
iteration 48 99.99999999999964 5 99.99999999999929 100
iteration 49 99.99999999999983 5 99.99999999999964 100
iteration 50 99.99999999999991 5 99.99999999999983 100
num selected antennas 5
2.3371280470123166
(8, 8, 4), eg: 25
iteration 1 50.0000005 5 1e-06 100
iteration 2 75.00000025 5 50.0000005 100
iteration 3 87.500000125 5 75.00000025 100
iteration 4 93.75000006249999 5 87.500000125 100
iteration 5 96.87500003125 5 93.75000006249999 100
iteration 6 98.437500015625 5 96.87500003125 100
iteration 7 99.2187500078125 5 98.437500015625 100
iteration 8 99.60937500390625 5 99.2187500078125 100
iteration 9 99.80

iteration 40 99.99999999990905 5 99.9999999998181 100
iteration 41 99.99999999995453 5 99.99999999990905 100
iteration 42 99.99999999997726 5 99.99999999995453 100
iteration 43 99.99999999998863 5 99.99999999997726 100
iteration 44 99.99999999999432 5 99.99999999998863 100
iteration 45 99.99999999999716 5 99.99999999999432 100
iteration 46 99.99999999999858 5 99.99999999999716 100
iteration 47 99.99999999999929 5 99.99999999999858 100
iteration 48 99.99999999999964 5 99.99999999999929 100
iteration 49 99.99999999999983 5 99.99999999999964 100
iteration 50 99.99999999999991 5 99.99999999999983 100
num selected antennas 5
3.487209535727879
(8, 8, 4), eg: 29
iteration 1 50.0000005 5 1e-06 100
iteration 2 75.00000025 5 50.0000005 100
iteration 3 87.500000125 5 75.00000025 100
iteration 4 93.75000006249999 5 87.500000125 100
iteration 5 96.87500003125 5 93.75000006249999 100
iteration 6 98.437500015625 5 96.87500003125 100
iteration 7 99.2187500078125 5 98.437500015625 100
iteration 8 99.60

NameError: name 'solve_instances' is not defined

In [ ]:
    with Pool(num_egs) as p:
        out_oracle = p.map(solve_bb_pool, arguments_oracle)
        print('pool ended')
    
    optimal_solution_list = [out_oracle[i][0] for i in range(len(out_oracle))]
    optimal_objective_list = [out_oracle[i][1] for i in range(len(out_oracle))]
    oracle_time = np.mean([out_oracle[i][3] for i in range(len(out_oracle))])

    data = (instances, optimal_solution_list, optimal_objective_list, oracle_time)
    with open(result_filepath, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)    

    ########### LOAD data ################################################

    with open(result_filepath, 'rb') as handle:
        data = pickle.load(handle)
        instances, optimal_solution_list, optimal_objective_list, oracle_time = data

    ########## OMAR'S Method #######################################
    mask_omar = []
    omar_time = 0
    for i in range(num_egs):
        H = instances[i,0,:,:] + 1j*instances[i,1,:,:]
        obj_bb += optimal_objective_list[i]
        t1 = time.time()
        obj, mask = as_omar(H, max_ant=max_ant)
        omar_time += time.time() - t1
        obj_omar += obj
        mask_omar.append(mask)
    omar_time = omar_time/num_egs
    print(omar_time)


In [5]:
len(combinations)

40